In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn 
import imblearn
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, balanced_accuracy_score, precision_recall_fscore_support, roc_auc_score

In [ ]:
#Ignorando avisos
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Formatação
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold= 15)
np.set_printoptions(precision=3)
sns.set(style="darkgrid")
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [ ]:
df = pd.read_csv('C_Dados_V5.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['ts'].unique()

In [ ]:
df['date'].unique()

In [ ]:
df['time'].unique()

In [ ]:
df['fridge_temperature'].unique()

In [ ]:
df['temp_condition'].unique()

In [ ]:
df['door_state'].unique()

In [ ]:
df['sphone_signal'].unique()

In [ ]:
df['latitude'].unique()

In [ ]:
df['longitude'].unique()

In [ ]:
df['FC1_Read_Input_Register'].unique()

In [ ]:
df['FC2_Read_Discrete_Value'].unique()

In [ ]:
df['FC3_Read_Holding_Register'].unique()

In [ ]:
df['FC4_Read_Coil'].unique()

In [ ]:
df['motion_status'].unique()

In [ ]:
df['light_status'].unique()

In [ ]:
df['current_temperature'].unique()

In [ ]:
df['thermostat_status'].unique()

In [ ]:
df['temperature'].unique()

In [ ]:
df['pressure'].unique()

In [ ]:
df['humidity'].unique()

In [ ]:
df['label'].unique()

In [ ]:
df['type'].unique()

## Processamento dos Dados
#### Dimensionamento (StandardScaler / Padronização)

Antes de fazer o preprocessamento : 
- Dividir o conjunto (evitando o vazamento' de informação durante cada etapa do processo).

In [ ]:
df

In [ ]:
# Teste sem a feature de TS. 
df.drop(['ts', 'type'], axis=1, inplace=True)

# Substituindo os espaços em branco na coluna 'time'
df['time'] = df['time'].str.replace(' ', '')

df['hour'] = ''
df['minute'] = ''
df['second'] = ''

df[['hour', 'minute', 'second']] = df['time'].str.split(':', expand=True)

df['hour'] = df['hour'].astype(int)
df['minute'] = df['minute'].astype(int)
df['second'] = df['second'].astype(int)


# Criando outras features usando a data (day-month-year)
df.date = pd.to_datetime(df.date)
df['day'] = df.date.dt.day
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year

df.drop(labels=['date'], inplace=True, axis=1)
df.head()

In [ ]:
#Excluindo também a coluna 'year', porque ela só contém um valor. 
df.drop(['year','month', 'time'], axis=1, inplace=True)
#Head()
df.head()

In [ ]:
df['label'].value_counts()

In [ ]:
df['label'].value_counts()/df.shape[0]

In [ ]:
# Separando as Features do Label
y_data = df.label
X_data = df.drop(['label'], axis=1)

### Pipeline

O bloco de código a seguir realiza pré-processamento do conjunto de dados, com o objetivo de prepará-lo para treinar um modelo de aprendizado de máquina.

A primeira linha define uma lista com o nome das colunas que contêm variáveis categóricas.

Em seguida, é criado um objeto `ColumnTransformer` que irá lidar com os dados das colunas categóricas, usando a classe OrdinalEncoder para transformar esses dados em valores numéricos ordinais. O parâmetro **``remainder='passthrough'``** é usado para manter as colunas que não são categóricas inalteradas.

Por fim, criamo um objeto `Pipeline` que irá aplicar o pré-processamento aos dados. Ele consiste em dois passos:

* O primeiro passo é aplicar o ColumnTransformer criado anteriormente, que irá lidar com as colunas categóricas e manter as outras colunas sem modificação.
* O segundo passo é aplicar um StandardScaler para normalizar os valores das colunas numéricas.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

categorical_features = ['temp_condition', 'door_state', 'sphone_signal', 'light_status', 'thermostat_status', ]
cat_handle = ColumnTransformer(
                    transformers=[
                        ('cat', OrdinalEncoder(), categorical_features),
                         ], remainder='passthrough')

# Juntamos tudo; Lidando com os dados categoricos e em seguida fazendo o standardscaler
preprocessor = Pipeline(steps=[
    ('categorical', cat_handle), 
    ('numerical', StandardScaler())
])

## Divisão dos dados

Esse código está usando o objeto StratifiedShuffleSplit da biblioteca sklearn.model_selection para dividir um conjunto de dados em conjuntos de treino e teste. A divisão é feita de forma estratificada, ou seja, preservando a proporção de cada classe do conjunto de dados original nos conjuntos de treino e teste.

A classe StratifiedShuffleSplit é uma estratégia de validação cruzada que, ao contrário da validação cruzada tradicional, não faz uma partição fixa do conjunto de dados em k conjuntos. Em vez disso, ela faz várias partições aleatórias do conjunto de dados e, em cada uma delas, mantém a proporção de cada classe nos conjuntos de treino e teste. Essa abordagem é útil quando o conjunto de dados é desbalanceado, ou seja, quando algumas classes têm muito mais instâncias do que outras.

O objeto StratifiedShuffleSplit é inicializado com três parâmetros:

* `n_splits`: número de partições a serem geradas. Neste caso, é gerada apenas uma partição.
* `test_size`: proporção do conjunto de dados a ser usada como teste. Neste caso é default, usamos uma proporção de 0.2, o que significa que 20% das instâncias são usadas como teste.
*`random_state`: semente para o gerador de números aleatórios. Neste caso, é usada a semente 0.

O loop for é usado para iterar sobre a única partição gerada pelo objeto StratifiedShuffleSplit. Em cada iteração, ele recebe os índices das instâncias que serão usadas como treino e teste e cria dois novos conjuntos de dados (*X_train, y_train* e *X_test, y_test*) com essas instâncias. Esses conjuntos de dados são usados posteriormente para treinar e testar um modelo de aprendizado de máquina.

**Em resumo, esse código é uma forma de dividir um conjunto de dados em conjuntos de treino e teste de forma estratificada, o que pode ser útil quando o conjunto de dados é desbalanceado.**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, 
                                                    shuffle=True, 
                                                    random_state=42, 
                                                    stratify=y_data)

# Reparar que está sendo usado o X_data e y_data sem passar o transform neles ainda.

# Regressão Logística

Foi adicionado algumas configurações no modelo de Regressão Logística, como max_iter, solver, C, e penalty. Além disso, o GridSearchCV é utilizado para ajustar nossos parâmetros em uma grade de valores e encontrar a melhor combinação.

Existem alguns parâmetros que você pode ajustar para uma regressão logística no scikit-learn. Aqui estão alguns exemplos:

* **`penalty`**: Especifica a norma a ser usada na regularização. Pode ser 'L1', 'L2', 'elasticnet' ou 'none'.


* **`C`**: Parâmetro de inversão de regularização. Valores menores especificam uma regularização mais forte.


* **`solver`**: Algoritmo a ser usado no problema de otimização. Pode ser 'newton-cg', 'lbfgs', 'liblinear', 'sag' ou 'saga'.


* **`max_iter`**: Número máximo de iterações para o solucionador convergir.


* **`multi_class`**: Especifica o esquema de classificação multiclasse. Pode ser 'ovr' (one-vs-rest) ou 'multinomial'.


* **`class_weight`**: Peso atribuído a cada classe. Pode ser 'balanced' ou um dicionário com pesos personalizados.

In [ ]:
from sklearn.feature_selection import SelectFromModel

# A variavel 'pipeline', contem a etapa de preprocesamento e o modelo, além do feature selection
pipeline = Pipeline(steps=[
    ('preprocessamento', preprocessor),
    ('feature_selection', SelectFromModel(estimator=RandomForestClassifier() , max_features=3)),
    ('classificador', LogisticRegression())
])

param_grid_LR = {
                'feature_selection__max_features': [1,2,3],
                'classificador__penalty': ['l1', 'l2'],
                'classificador__C': [0.1, 1.0, 10.0],
                'classificador__solver': ['newton-cg', 'saga'],
                'classificador__max_iter': [100, 1000],
                'classificador__multi_class': ['ovr', 'multinomial'],
                'classificador__class_weight': ['balanced']}

# Repare que o param_grid é passado o nome do classifier dois underscore 
# antes do parametro assim: nomeclassificador__parametro. O nome é passado na string do pipeline
# no caso deixei 'classificador' mesmo.
grid_search_RL = GridSearchCV(pipeline, param_grid=param_grid_LR, cv=5, n_jobs=-1, refit=True)

A função GridSearchCV da biblioteca sklearn.model_selection possui diversos parâmetros que podem ser utilizados para controlar o processo de busca de hiperparâmetros e a validação cruzada. Abaixo estão listados os principais parâmetros:

* **`estimator`**: representa o modelo a ser otimizado e deve ser uma instância de um estimador do scikit-learn.


* **`param_grid`**: um dicionário que mapeia nomes de parâmetros do modelo para listas de valores a serem explorados durante a busca de hiperparâmetros.


* **`scoring`**: uma métrica de avaliação que será utilizada para avaliar o desempenho do modelo. Deve ser uma string que representa o nome da métrica ou uma função que calcula a métrica. Por padrão, é utilizado o score da função score() do estimador.


* **`cv`**: número de partições a serem utilizadas na validação cruzada.


* **`n_jobs`**: número de trabalhos em paralelo a serem executados. Se n_jobs=-1, todos os processadores disponíveis serão utilizados.


* **`verbose`**: nível de verbosidade do output.


* **`pre_dispatch`**: número de trabalhos que devem ser despachados para o trabalhador antes que o próximo lote de tarefas seja despachado. O valor padrão é 2 * n_jobs.


* **`return_train_score`**: se True, inclui o score de treino para cada combinação de parâmetros no resultado. O valor padrão é False.


* **`refit`**: se True, refita o modelo com os melhores parâmetros encontrados usando todos os dados disponíveis. O valor padrão é True.


* **`iid`**: se True, assume que as dobras de validação cruzada são independentes e identicamente distribuídas (i.i.d.), o que não é garantido para todos os tipos de dados. O valor padrão é True.


* **`error_score`**: valor a ser atribuído ao score caso ocorra algum erro na validação cruzada.


* **`return_estimator`**: se True, retorna os estimadores que foram ajustados para cada combinação de parâmetros. O valor padrão é False.

In [ ]:
# fit
grid_search_RL.fit(X_train, y_train)

In [ ]:
# Exibindo os melhores parâmetros
print(grid_search_RL.best_params_)

In [ ]:
# Essas métricas são do conjunto de validação (é pra ver como o modelo se comportou) 
# O std_score é o desvio padrão (O valor tende a ser baixo)
# Não é necessário mostrar métricas de valição ou treino. 
# O que importa é a metrica no TESTE.
# O gridsearch faz a validação cruzado k-fold, o cv=5 são 5 folds.
index = grid_search_RL.best_index_
results = grid_search_RL.cv_results_

mean_score = results['mean_test_score'][index]
std_score  = results['std_test_score'][index]

print(f"Validation score: {mean_score:.5f} +- {std_score:.5f}")

In [ ]:
# Aqui é realizada a predição.
# O gridsearch possui um paramentro chamado refit 
#quando eles está true quer dizer que o modelo JÁ É treinado com os melhores parametros, por isso já dou um predict direto

y_pred = grid_search_RL.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

# Classification Report (Apenas dos dados de teste)
print(classification_report(y_test, y_pred))

In [ ]:
# matrix de confusão. Apenas do teste
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, 
                                        display_labels=['Normal', 'Ataque'], 
                                        #normalize = 'true', values_format='.1%',
                                        cmap=plt.cm.Blues, colorbar=False
                                        )
plt.show()

# KNeighborsClassifier

Existem vários parâmetros que podem ser ajustados para o modelo KNN usando a busca em grade (GridSearchCV). Esses são alguns dos principais parâmetros que podem ser incluídos no dicionário param_grid:

* **`n_neighbors`**: Número de vizinhos mais próximos a serem considerados. É um parâmetro obrigatório do modelo KNN. 


* **`weights`**: Método de ponderação dos vizinhos próximos. Os valores possíveis são "uniform" (ponderação uniforme) ou "distance" (ponderação pela inversa da distância).


* **`algorithm`**: Algoritmo a ser usado para encontrar os vizinhos próximos. Os valores possíveis são "auto" (o algoritmo escolhe o mais apropriado com base nos dados), "ball_tree" (utiliza uma estrutura de dados de árvore para acelerar a busca) ou "kd_tree" (utiliza uma estrutura de dados de árvore k-dimensionais para acelerar a busca).


* **`leaf_size`**: Tamanho da folha a ser usado na estrutura de dados da árvore. Isso afeta a velocidade e a memória necessárias para construir a árvore.


* **`p`**: Parâmetro de potência a ser usado na métrica de distância de Minkowski. Se p=1, a distância de Manhattan é usada. Se p=2, a distância euclidiana é usada.


* **`metric`**: Métrica de distância a ser usada para medir a distância entre os pontos. Os valores possíveis são "euclidean", "manhattan", "chebyshev", "minkowski" (usado com o parâmetro p) e outras métricas personalizadas.

In [ ]:
from sklearn.feature_selection import SelectFromModel

pipeline_knn = Pipeline(steps=[
    ('preprocessamento', preprocessor),
    ('feature_selection', SelectFromModel(estimator=RandomForestClassifier() , max_features=3)),
    ('classificador', KNeighborsClassifier())
])

param_grid_KNN = {
                'feature_selection__max_features': [1,2,3],
                'classificador__n_neighbors': [1, 3, 5, 7, 9],
                'classificador__metric': ['euclidean', 'manhatan', 'chebyshev', 'minkowski']}

grid_search = GridSearchCV(pipeline_knn, param_grid=param_grid_KNN, cv=5, n_jobs=-1, refit=True)

In [ ]:
# fit
grid_search.fit(X_train, y_train)

In [ ]:
# Exibindo os melhores parâmetros
print(grid_search.best_params_)

In [ ]:
index = grid_search.best_index_
results = grid_search.cv_results_

mean_score = results['mean_test_score'][index]
std_score  = results['std_test_score'][index]

print(f"Validation score: {mean_score:.5f} +- {std_score:.5f}")

In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
# Matrix de Confusão (Apenas do teste)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, 
                                        display_labels=['Normal', 'Ataque'], 
                                        #normalize = 'true', values_format='.1%',
                                        cmap=plt.cm.Blues, colorbar=False
                                        )
plt.show()

## PipeLine Gradient Boosting Master

* **`learning_rate`**: Taxa de aprendizado do algoritmo.


* **`n_estimators`**: Número de estimadores no algoritmo.


* **`max_depth`**: Profundidade máxima das árvores de decisão.


* **`min_samples_split`**: Número mínimo de amostras necessárias para dividir um nó interno.


* **`min_samples_leaf`**: Número mínimo de amostras necessárias em uma folha.


* **`max_features`**: Número máximo de recursos considerados para dividir um nó.


* **`subsample`**: Fração de amostras usadas para treinar cada árvore.

In [ ]:
pipeline_gbm = Pipeline(steps=[
    ('preprocessamento', preprocessor),
    ('feature_selection', SelectFromModel(estimator=RandomForestClassifier() , max_features=3)),
    ('classificador', GradientBoostingClassifier())
])

param_grid_GBM = {
    'feature_selection__max_features': [1,2,3],
    'classificador__learning_rate': [0.1, 0.05],
    'classificador__n_estimators': [50, 100],
    'classificador__max_depth': [2, 3],
    'classificador__min_samples_split': [2, 4],
    'classificador__min_samples_leaf': [1, 2],
    'classificador__max_features': ['auto', 'sqrt'],
    'classificador__subsample': [0.8, 1.0]}


In [ ]:
# fit
grid_search.fit(X_train, y_train)

In [ ]:
# Exibindo os melhores parâmetros
print(grid_search.best_params_)

In [ ]:
index = grid_search.best_index_
results = grid_search.cv_results_

mean_score = results['mean_test_score'][index]
std_score  = results['std_test_score'][index]

print(f"Validation score: {mean_score:.5f} +- {std_score:.5f}")

In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
# Matrix de Confusão (Apenas do teste)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, 
                                        display_labels=['Normal', 'Ataque'], 
                                        #normalize = 'true', values_format='.1%',
                                        cmap=plt.cm.Blues, colorbar=False
                                        )
plt.show()

# Nayve Bayes BernoulliNB

A função BernoulliNB() tem apenas um hiperparâmetro para ajuste:

* `alpha`: parâmetro de suavização Laplace. Quanto maior o valor de alpha, maior é a suavização aplicada.

In [ ]:
pipeline_nb = Pipeline(steps=[
    ('preprocessamento', preprocessor),
    ('feature_selection', SelectFromModel(estimator=RandomForestClassifier() , max_features=3)),
    ('classificador', BernoulliNB())
])

param_grid_NB = {
    'feature_selection__max_features': [1,2,3],
    'classificador__alpha': [0.1, 0.5, 1.0]}

grid_search = GridSearchCV(pipeline_nb, param_grid=param_grid_NB, cv=5, n_jobs=-1, refit=True)

In [ ]:
# fit
grid_search.fit(X_train, y_train)

In [ ]:
# Exibindo os melhores parâmetros
print(grid_search.best_params_)

In [ ]:
index = grid_search.best_index_
results = grid_search.cv_results_

mean_score = results['mean_test_score'][index]
std_score  = results['std_test_score'][index]

print(f"Validation score: {mean_score:.5f} +- {std_score:.5f}")

In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
# Classification Report (Apenas dos dados de teste)
print(classification_report(y_test, y_pred))

In [ ]:
# Matrix de Confusão (Apenas do teste)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, 
                                        display_labels=['Normal', 'Ataque'], 
                                        #normalize = 'true', values_format='.1%',
                                        cmap=plt.cm.Blues, colorbar=False
                                        )
plt.show()

# PipeLine Linear Discriminant Analysis

Aqui estão os principais parâmetros que podemos ajustar:

* **`solver`**: Algoritmo usado para encontrar a solução. Possíveis valores são svd, lsqr ou eigen.


* **`shrinkage`**: Parâmetro de encolhimento utilizado para melhorar a estabilidade da estimativa. Possíveis valores são None, auto ou um valor float entre 0 e 1.


* **`tol`**: Tolerância para a convergência do algoritmo. Padrão é 1e-4.


* **`n_components`**: Número de componentes para manter. O padrão é manter todas as componentes.


* **`priors`**: Probabilidades a priori de cada classe. Se None, as probabilidades são ajustadas de acordo com os dados.


* **`store_covariance`**: Se verdadeiro, armazena a matriz de covariância empírica de cada classe. Padrão é False.

In [ ]:
pipeline_lda = Pipeline(steps=[
    ('preprocessamento', preprocessor),
    ('feature_selection', SelectFromModel(estimator=RandomForestClassifier() , max_features=3)),
    ('classificador', LinearDiscriminantAnalysis())
])

param_grid_LDA = {
    'feature_selection__max_features': [1,2,3],
    'classificador__solver': ['svd', 'lsqr', 'eigen']}

grid_search = GridSearchCV(pipeline_lda, param_grid=param_grid_LDA, cv=5, n_jobs=-1, refit=True)

In [ ]:
# fit
grid_search.fit(X_train, y_train)

In [ ]:
# Exibindo os melhores parâmetros
print(grid_search.best_params_)

In [ ]:
index = grid_search.best_index_
results = grid_search.cv_results_

mean_score = results['mean_test_score'][index]
std_score  = results['std_test_score'][index]

print(f"Validation score: {mean_score:.5f} +- {std_score:.5f}")

In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
# Classification Report (Apenas dos dados de teste)
print(classification_report(y_test, y_pred))

In [ ]:
# Matrix de Confusão (Apenas do teste)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, 
                                        display_labels=['Normal', 'Ataque'], 
                                        #normalize = 'true', values_format='.1%',
                                        cmap=plt.cm.Blues, colorbar=False
                                        )
plt.show()

# DecisionTreeClassifier

Aqui estão alguns dos parâmetros mais comuns que podemos incluir no param_grid para o DecisionTreeClassifier:

* **`criterion`**: critério de divisão usado na árvore de decisão. As opções são gini ou entropy.


* **`max_depth`**: profundidade máxima da árvore de decisão. Um valor mais alto permitirá que a árvore de decisão tenha mais níveis, o que pode levar a uma melhor precisão, mas também pode levar a um maior risco de sobreajuste.


* **`min_samples_split`**: o número mínimo de amostras necessárias para dividir um nó. Isso ajuda a evitar divisões que levam a subárvores muito pequenas.


* **`min_samples_leaf`**: o número mínimo de amostras necessárias em uma folha. Isso ajuda a evitar folhas que contenham muito poucas amostras.


* **`max_features`**: o número máximo de recursos a serem considerados para cada divisão. Isso pode ajudar a reduzir o risco de sobreajuste.


* **`class_weight`**: pesos associados a cada classe. Isso pode ser útil para lidar com conjuntos de dados desbalanceados.

In [ ]:
pipeline_dtc = Pipeline(steps=[
    ('preprocessamento', preprocessor),
    ('feature_selection', SelectFromModel(estimator=RandomForestClassifier() , max_features=3)),
    ('classificador', DecisionTreeClassifier())
])

param_grid_DTC = {
    'feature_selection__max_features': [1,2,3],
    'classificador__criterion': ['gini', 'entropy'],
    'classificador__max_depth': [2, 4, 6],
    'classificador__min_samples_split': [2, 5, 10],
    'classificador__min_samples_leaf': [1, 2, 4],
    'classificador__max_features': ['sqrt', 'log2'],
    'classificador__class_weight': [None, 'balanced']}

grid_search = GridSearchCV(pipeline_dtc, param_grid=param_grid_DTC, cv=5, n_jobs=-1, refit=True)

In [ ]:
# fit
grid_search.fit(X_train, y_train)

In [ ]:
# Exibindo os melhores parâmetros
print(grid_search.best_params_)

In [ ]:
index = grid_search.best_index_
results = grid_search.cv_results_

mean_score = results['mean_test_score'][index]
std_score  = results['std_test_score'][index]

print(f"Validation score: {mean_score:.5f} +- {std_score:.5f}")

In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
# Classification Report (Apenas dos dados de teste)
print(classification_report(y_test, y_pred))

In [ ]:
# Matrix de Confusão (Apenas do teste)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, 
                                        display_labels=['Normal', 'Ataque'], 
                                        #normalize = 'true', values_format='.1%',
                                        cmap=plt.cm.Blues, colorbar=False
                                        )
plt.show()